<a href="https://colab.research.google.com/github/yuuiwqy622/unsupervised-segmentation/blob/main/unsupervised_segmentatation_isic2018_fid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on https://blockgeni.com/how-to-implement-the-frechet-inception-distance-fid-for-evaluating-gans/

In [ ]:
!pip install dominate

In [ ]:
!git clone https://github.com/NVIDIA/pix2pixHD.git

In [ ]:
!./generate-images.sh

In [ ]:
# example of calculating the frechet inception distance in Keras
import numpy as np
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import shuffle
from numpy.random import randint
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize
import time
from PIL import Image
from glob import glob

In [ ]:
# scale an array of images to a new size
def scale_images(images, new_shape):
  images_list = []
  for image in images:
    # resize with nearest neighbor interpolation
    new_image = resize(image, new_shape, 0)
    # store
    images_list.append(new_image)
  return asarray(images_list)
 
# calculate frechet inception distance
def calculate_fid(model, images1, images2):
  # calculate activations
  act1 = model.predict(images1)
  act2 = model.predict(images2)
  # calculate mean and covariance statistics
  mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
  mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
  # calculate sum squared difference between means
  ssdiff = numpy.sum((mu1 - mu2)**2.0)
  # calculate sqrt of product between cov
  covmean = sqrtm(sigma1.dot(sigma2))
  # check and correct imaginary numbers from sqrt
  if iscomplexobj(covmean):
   covmean = covmean.real
  # calculate score
  fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
  return fid

In [ ]:
def load_images(files: list):
  ims = []
  for f in files:
    ims.append(np.asarray(Image.open(f)))
  return np.asarray(ims)

In [ ]:
start = time.time()
fid_scores = []

for epoch in range(10, 101, 10):
  print(f'epoch: {epoch}')

  files = glob(f'pix2ultra/results/pix2ultra/test_{epoch}/images/*_synthesized_image.jpg')
  synth = load_images(files)
  files = glob('isic-train-100/*.jpg')
  orig = load_images(files)
  print('Loaded', synth.shape, orig.shape)

  model = InceptionV3(include_top=False, pooling='avg', input_shape=(256,256,3))

  synth = synth.astype('float32')
  orig = orig.astype('float32')

  synth = preprocess_input(synth)
  orig = preprocess_input(orig)

  fid = calculate_fid(model, synth, orig)
  print('FID (different): %.3f' % fid)
  fid_scores.append((epoch, fid))

fid_scores = sorted(fid_scores, key=lambda s: s[1])
best_epoch, best_fid = fid_scores[0]
print(f'Best epoch is {best_epoch} with FID: {best_fid}')
end = time.time()
print(f'Duration: {end-start} s')

epoch: 10
Loaded (100, 256, 256, 3) (100, 256, 256, 3)
FID (different): 360.746
epoch: 20
Loaded (100, 256, 256, 3) (100, 256, 256, 3)
FID (different): 320.621
epoch: 30
Loaded (100, 256, 256, 3) (100, 256, 256, 3)
FID (different): 314.935
epoch: 40
Loaded (100, 256, 256, 3) (100, 256, 256, 3)
FID (different): 321.539
epoch: 50
Loaded (100, 256, 256, 3) (100, 256, 256, 3)
FID (different): 250.566
epoch: 60
Loaded (100, 256, 256, 3) (100, 256, 256, 3)
FID (different): 246.002
epoch: 70
Loaded (100, 256, 256, 3) (100, 256, 256, 3)
FID (different): 207.718
epoch: 80
Loaded (100, 256, 256, 3) (100, 256, 256, 3)
FID (different): 191.612
epoch: 90
Loaded (100, 256, 256, 3) (100, 256, 256, 3)
FID (different): 192.970
epoch: 100
Loaded (100, 256, 256, 3) (100, 256, 256, 3)
FID (different): 191.929
Best epoch is 80 with FID: 191.61154224422694
Duration: 146.9808473587036 s
